# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset, Model, Datastore, Environment
from azureml.data.datapath import DataPath
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import pandas as pd
import joblib, pickle

## Dataset

### Overview
The dataset I'm using tracks car mpg as a function of several car characteristics like weight and acceleration. The task performed is regression to predict mpg as a function of these characteristics.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
subscription_id = '61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30'
resource_group = 'aml-quickstarts-133182'
workspace_name = 'quick-starts-ws-133182'

ws = Workspace.get(name="quick-starts-ws-133182")
exp = Experiment(workspace=ws, name="Ranga")

workspace = Workspace(subscription_id, resource_group, workspace_name)

CEdata = Dataset.get_by_name(workspace, name='CE')
CEdata.download(target_path='.', overwrite=True)
VS1data = Dataset.get_by_name(workspace, name='VS1')
VS1data.download(target_path='.', overwrite=True)
TS4data = Dataset.get_by_name(workspace, name='TS4')
TS4data.download(target_path='.', overwrite=True)
TS3data = Dataset.get_by_name(workspace, name='TS3')
TS3data.download(target_path='.', overwrite=True)



['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ranganext/code/Users/odl_user_133182/TS3.txt']

In [3]:
SEdata = Dataset.get_by_name(workspace, name='SE')
SEdata.download(target_path='.', overwrite=True)
PS6data = Dataset.get_by_name(workspace, name='PS6')
PS6data.download(target_path='.', overwrite=True)
PS5data = Dataset.get_by_name(workspace, name='PS5')
PS5data.download(target_path='.', overwrite=True)

FS2data = Dataset.get_by_name(workspace, name='FS2')
FS2data.download(target_path='.', overwrite=True)

EPS1data = Dataset.get_by_name(workspace, name='EPS1')
EPS1data.download(target_path='.', overwrite=True)
CPdata = Dataset.get_by_name(workspace, name='CP')
CPdata.download(target_path='.', overwrite=True)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ranganext/code/Users/odl_user_133182/CP.txt']

In [4]:
cooleff = []
coolpwr = []

ps_five = []
ps_six = []
motorpower = []

volflowtwo = []

tempthree = []
tempfour = []
effactor = []
vibes = []

In [5]:
ce = open('CE.txt','r')
for line in ce:
    cooleff.append(line.split())

In [6]:
cp = open('CP.txt','r')
for line in cp:
    coolpwr.append(line.split())

effic = open('se.txt','r')
for line in effic:
    effactor.append(line.split())
    
vibe = open('VS1.txt')
for line in vibe:
    vibes.append(line.split())

In [7]:
psfive = open('PS5.txt','r')
for line in psfive:
    ps_five.append(line.split())
pssix = open('PS6.txt','r')
for line in pssix:
    ps_six.append(line.split())

In [8]:
mp = open('EPS1.txt','r')
for line in mp:
    motorpower.append(line.split())


tthree = open('TS3.txt','r')
for line in tthree:
    tempthree.append(line.split())
tfour = open('TS4.txt','r')
for line in tfour:
    tempfour.append(line.split())

In [9]:
vftwo = open('FS2.txt','r')
for line in vftwo:
    volflowtwo.append(line.split())

In [10]:
cooleff = np.reshape(cooleff,(2205,60))
coolpwr = np.reshape(coolpwr,(2205,60))
cooleffDF = pd.DataFrame(cooleff)
coolpwrDF = pd.DataFrame(coolpwr)

In [11]:
ps_five = np.reshape(ps_five,(2205,6000))
ps_five_DF = pd.DataFrame(ps_five)
ps_six = np.reshape(ps_six,(2205,6000))
ps_six_DF = pd.DataFrame(ps_six)

In [12]:
motorpower = np.reshape(motorpower,(2205,6000))
mp_DF = pd.DataFrame(motorpower)

volflowtwo = np.reshape(volflowtwo,(2205,600))
vftwo_DF = pd.DataFrame(volflowtwo)

In [13]:
tempthree = np.reshape(tempthree,(2205,60))
tthree_DF = pd.DataFrame(tempthree)
tempfour = np.reshape(tempfour,(2205,60))
tfour_DF = pd.DataFrame(tempfour)


In [14]:
effactor = np.reshape(effactor,(2205,60))
eff_DF = pd.DataFrame(effactor)
vibes = np.reshape(vibes,(2205,60))
vibes_DF = pd.DataFrame(vibes)

In [15]:
coolpwrDF = coolpwrDF.astype(float)
coolpwrDF['Avg coolpwr'] = coolpwrDF.mean(axis=1)
coolpwrDF = pd.DataFrame(coolpwrDF.loc[:,'Avg coolpwr'])

cooleffDF = cooleffDF.astype(float)
cooleffDF['Avg cooleff'] = cooleffDF.mean(axis=1)
cooleffDF = pd.DataFrame(cooleffDF.loc[:,'Avg cooleff'])

In [16]:
ps_five_DF = ps_five_DF.astype(float)
ps_five_DF['Avg psfive'] = ps_five_DF.mean(axis=1)
ps_five_DF = pd.DataFrame(ps_five_DF.loc[:,'Avg psfive'])

ps_six_DF = ps_six_DF.astype(float)
ps_six_DF['Avg pssix'] = ps_six_DF.mean(axis=1)
ps_six_DF = pd.DataFrame(ps_six_DF.loc[:,'Avg pssix'])

In [17]:
mp_DF = mp_DF.astype(float)
mp_DF['Avg mp'] = mp_DF.mean(axis=1)
mp_DF = pd.DataFrame(mp_DF.loc[:,'Avg mp'])

vftwo_DF = vftwo_DF.astype(float)
vftwo_DF['Avg vftwo'] = vftwo_DF.mean(axis=1)
vftwo_DF = pd.DataFrame(vftwo_DF.loc[:,'Avg vftwo'])

In [18]:
eff_DF = eff_DF.astype(float)
eff_DF['Avg eff'] = eff_DF.mean(axis=1)
eff_DF = pd.DataFrame(eff_DF.loc[:,'Avg eff'])

vibes_DF = vibes_DF.astype(float)
vibes_DF['Avg vibes'] = vibes_DF.mean(axis=1)
vibes_DF = pd.DataFrame(vibes_DF.loc[:,'Avg vibes'])

In [19]:
tthree_DF = tthree_DF.astype(float)
tthree_DF['Avg tthree'] = tthree_DF.mean(axis=1)
tthree_DF = pd.DataFrame(tthree_DF.loc[:,'Avg tthree'])
tfour_DF = tfour_DF.astype(float)
tfour_DF['Avg tfour'] = tfour_DF.mean(axis=1)
tfour_DF = pd.DataFrame(tfour_DF.loc[:,'Avg tfour'])

In [20]:
features = (((((((((coolpwrDF.join(cooleffDF)).join(ps_five_DF)).join(ps_six_DF)).join(mp_DF)).join(vftwo_DF)).join(eff_DF)).join(vibes_DF)).join(tthree_DF)).join(tfour_DF))

In [21]:
profiledata = Dataset.get_by_name(workspace, name='profile')
profiledata.download(target_path='.', overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/ranganext/code/Users/odl_user_133182/profile.txt']

In [29]:
profiles=[]
prof = open('profile.txt','r')
for line in prof:
    profiles.append(line.split())

labels = np.reshape(profiles,(2205,5))
labels_DF = pd.DataFrame(labels)


labels_DF.columns = ['Clr cndn','Valve cndn','Pump lkg','Hydraulic press','Stable Flag (only cat)']
labels_DF = labels_DF.astype(float)

In [30]:
ds = features.join(labels_DF)

## AutoML Configuration

I chose the regression task since that represents the nature of this problem, and R-squared is correspondingly a good metric. R-squared is also the default scoring metric for GradientBoosting Regressor, which is the model that I'm using in Hyperparameter tuning.

In [31]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='regression',
    primary_metric='r2_score',
    training_data=ds,
    label_column_name='Valve cndn',
    n_cross_validations=3)

In [32]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run = autoexp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary de

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:31       0.8811    0.8811
         1   MaxAbsScaler XGBoostRegressor                  0:00:25       0.8008    0.8811
         2   SparseNormalizer XGBoostRegressor              0:00:27       0.7696    0.8811
         3   MaxAbsScaler RandomForest                      0:00:30       0.7959    0.8811
         4   StandardScalerWrapper XGBoostRegressor         0:00:27       0.8914    0.8914
         5   MaxAbsScaler ElasticNet                        0:00:25       0.4609    0.8914
         6   MinMaxScaler RandomForest                      0:00:34       0.8037    0.8914
         7   Standar

## Run Details

The models trained feature a variety of preprocessing methods i.e. scaling to 0-1, scaling by max values, and normalization. After this preparation, a variety of models are fit. What we find is that simple stochastic gradient descents and decision trees are largely inadequate. Random Forest uses many more trees (the 'bagging' technique) with many leaves of equal weight. Not surprisingly, adding 'boosting' to bagging through XGBoost results in improved performance since we penalize leaves that don't improve model performance.
TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [6]:
best,fitted = remote_run.get_output()
print(best)

Run(Experiment: AutoRanga,
Id: AutoML_c1ff0474-a355-41c9-8bb5-3317faf8de83_45,
Type: None,
Status: Completed)


In [7]:
print(fitted)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [9]:
fitted.steps[-1]

('prefittedsoftvotingregressor',
 PreFittedSoftVotingRegressor(estimators=[('42',
                                           Pipeline(memory=None,
                                                    steps=[('sparsenormalizer',
                                                            <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7f479bd2d9b0>),
                                                           ('xgboostregressor',
                                                            XGBoostRegressor(base_score=0.5,
                                                                             booster='gbtree',
                                                                             colsample_bylevel=1,
                                                                             colsample_bynode=1,
                                                                             colsample_bytree=1,
                                                              

In [10]:
#TODO: Save the best model
filename = 'best_automl_model.sav'
# pickle.dump(fitted, open(filename, 'wb'))

joblib.dump(fitted,filename)

['best_automl_model.sav']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
registeredfinal = Model.register(workspace=ws,model_path='./best_automl_model.sav',model_name='registered.sav')

Registering model registered.sav


In [13]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-AutoML')

inferencefinal = InferenceConfig(entry_script='score.py',environment=env) 

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, AciWebservice


deployfinal = AciWebservice.deploy_configuration()
svcs = Model.deploy(workspace=ws,name='deploy',models=[registeredfinal],inference_config=inferencefinal,deployment_config=deployfinal)
svcs.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
print(svcs.state)

Healthy


In [15]:
print(svcs.scoring_uri)

http://29e036ea-6c5a-48e6-ad64-5464eb02ec6c.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
import requests
import json

# URL for the web service
scoring_uri = 'http://29e036ea-6c5a-48e6-ad64-5464eb02ec6c.southcentralus.azurecontainer.io/score'


data = {'data':[{
    'cylinders':4,
    'displacement':84,
    'horsepower':65,
    'weight':2110,
    'acceleration':12,
    'model yr':80,
    'origin':3}
]
        }

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}


# # Make the request and display the response
result = requests.post(scoring_uri,input_data,headers=headers)
print("prediction:",result.text)

prediction: [37.98189931616876]


In [18]:
env.save_to_directory('./training',overwrite=False)

TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
print(svcs.get_logs())

2020-12-05T14:57:54.8682742Z stdout F 2020-12-05T14:57:54,706579800+00:00 - iot-server/run 
2020-12-05T14:57:54.9463202Z stdout F 2020-12-05T14:57:54,784756300+00:00 - gunicorn/run 
2020-12-05T14:57:54.9752787Z stdout F 2020-12-05T14:57:54,812926400+00:00 - nginx/run 
2020-12-05T14:57:55.0473033Z stdout F 2020-12-05T14:57:54,813868900+00:00 - rsyslog/run 
2020-12-05T14:57:55.0743102Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-05T14:57:55.1577944Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-05T14:57:55.1618084Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-05T14:57:55.1618084Z stderr F /usr/sbin/nginx:

In [ ]:
svcs.delete()